# Predictive modelling

*Simple Probability Calculation at Nodes*

We want to calculate the gaussian distribution of the delays, at each stop in an hourly time interval. That is, for each stop and each hour between 7-17 we have a mean and standard deviation value for the delay. These distribution will then be used to define the certainty of sucess of a trip.

In [1]:
%%configure
{"conf": {
    "spark.app.name": "MVAY_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8639,application_1589299642358_3164,pyspark,busy,Link,Link,
8642,application_1589299642358_3167,pyspark,idle,Link,Link,
8693,application_1589299642358_3218,pyspark,idle,Link,Link,
8700,application_1589299642358_3225,pyspark,idle,Link,Link,
8702,application_1589299642358_3227,pyspark,idle,Link,Link,
8705,application_1589299642358_3230,pyspark,busy,Link,Link,
8707,application_1589299642358_3232,pyspark,busy,Link,Link,
8708,application_1589299642358_3233,pyspark,idle,Link,Link,
8709,application_1589299642358_3234,pyspark,idle,Link,Link,
8712,application_1589299642358_3237,pyspark,idle,Link,Link,


In [2]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8744,application_1589299642358_3269,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Import necessary packages
import pyspark.sql.functions as f
import math
from pyspark.sql.types import FloatType, IntegerType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Retrieve the stop_times_filt data

In [4]:
stop_times_filt = spark.read.orc("/user/fristedt/stop_times_filt.orc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Retrieve the relevant sbb data

In [5]:
# Load the entire sbb data
sbb_data = spark.read.orc("/data/sbb/orc/istdaten/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Keep only the relevant columns
sbb_data = sbb_data.select(f.col('bpuic').alias('stop_id'), f.col('haltestellen_name').alias('stop_name'), 
                           f.col('ankunftszeit').alias('arrival_time'), f.col('an_prognose').alias('arrival_time_prog'), 
                           f.col('an_prognose_status').alias('arr_time_prog_status'), f.col('produkt_id').alias('transport_type'))
sbb_data.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+----------------+-----------------+--------------------+--------------+
|stop_id|   stop_name|    arrival_time|arrival_time_prog|arr_time_prog_status|transport_type|
+-------+------------+----------------+-----------------+--------------------+--------------+
|8500090|Basel Bad Bf|                |                 |            PROGNOSE|           Zug|
|8500090|Basel Bad Bf|                |                 |            PROGNOSE|           Zug|
|8500090|Basel Bad Bf|03.09.2018 06:25|                 |           UNBEKANNT|           Zug|
+-------+------------+----------------+-----------------+--------------------+--------------+
only showing top 3 rows

We choose to only keep the rows where arr_time_prog_status is set to GESCHAETZT, PROGNOSE or REAL as these are the ones that we are interested in. We also remove the rows where there is missing values for both arrival_time as well as arrival_time_prog as we are unable to use these.

In [7]:
relevant_data = sbb_data.where(f.col('arr_time_prog_status').isin(['GESCHAETZT', 'PROGNOSE', 'REAL']))
relevant_data = relevant_data.where((relevant_data.arrival_time != '') & (relevant_data.arrival_time_prog != ''))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
relevant_data.cache()
relevant_data.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

767270257

### Join the dataframes

The stops from the filtered stop_times dataframe is joined with the relevant_data dataframe, such that we only compute the mean and standard deviation for the stops that are actually going to be in our directed graph.

In [9]:
# Retrieve the stop ids from the stop_times_filt df and join with the relevant_data df
ids = stop_times_filt.select('stop_id').distinct()
joined_df = ids.join(relevant_data, 'stop_id', 'inner')
joined_df.cache()

joined_df.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+----------------+-------------------+--------------------+--------------+
|stop_id|        stop_name|    arrival_time|  arrival_time_prog|arr_time_prog_status|transport_type|
+-------+-----------------+----------------+-------------------+--------------------+--------------+
|8502274|          Zufikon|03.09.2018 05:06|03.09.2018 05:07:04|                REAL|           Zug|
|8502275|         Heinrüti|03.09.2018 05:10|03.09.2018 05:10:27|                REAL|           Zug|
|8502268|Zufikon Belvédère|03.09.2018 05:11|03.09.2018 05:11:28|                REAL|           Zug|
+-------+-----------------+----------------+-------------------+--------------------+--------------+
only showing top 3 rows

In [10]:
joined_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

184071303

Went from having 767270257 rows in the relevant_data dataframe to having 193984287 rows in the joined df. The join was successful.

### Get the delay distributions

Now we calculate the delay for each stop at every given time. Then we extract the hour from the timestamps and filter such that we only have times between 7AM to 5PM. After this, we group by the hour and alongside compute the mean and standard deviation of the delay column.

#### Extract hour and filter for business hours

Since the timestamps are stored as strings, we can easily extract the hour from each scheduled arrival timestamp. We only keep the times that are within our time interval 7-17.

In [11]:
# Extract the hour from the scheduled arrival timestamp
hour_df = joined_df.withColumn('arrival_hour', joined_df.arrival_time.substr(12, 2).cast(IntegerType()))

# Filter such that we only keep times between 7-17
hour_df = hour_df.where((f.col('arrival_hour') >= 7) & (f.col('arrival_hour') <= 17))

hour_df.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+----------------+-------------------+--------------------+--------------+------------+
|stop_id|        stop_name|    arrival_time|  arrival_time_prog|arr_time_prog_status|transport_type|arrival_hour|
+-------+-----------------+----------------+-------------------+--------------------+--------------+------------+
|8502274|          Zufikon|03.09.2018 07:06|03.09.2018 07:06:58|                REAL|           Zug|           7|
|8502275|         Heinrüti|03.09.2018 07:10|03.09.2018 07:10:22|                REAL|           Zug|           7|
|8502268|Zufikon Belvédère|03.09.2018 07:11|03.09.2018 07:12:00|                REAL|           Zug|           7|
+-------+-----------------+----------------+-------------------+--------------------+--------------+------------+
only showing top 3 rows

#### Calculate delays

By converting the string formatted timestamp to unix_timestamp we can easily calculate the delay. Note that there may be negative delays, which means that the transportation arrived before it was scheduled. We filter out delays that are very large or very small (more than an hour), as they don't belong to normal and may distort our distributions a lot.

In [12]:
# Convert timestamp in string format to unix_timestamp format
temp_df = hour_df.withColumn('scheduled', f.unix_timestamp(hour_df.arrival_time, 'dd.MM.yyy HH:mm'))
temp_df = temp_df.withColumn('actual', f.unix_timestamp(temp_df.arrival_time_prog, 'dd.MM.yyy HH:mm'))

# Calculate the delays
delay_df = temp_df.withColumn('minute_delay', (temp_df.actual - temp_df.scheduled)/60).drop(*['scheduled', 'actual'])

# Remove unreasonable delays that may distort the distributions
delay_df = delay_df.where((f.col('minute_delay') < 60) & (f.col('minute_delay') > -60))

delay_df.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+----------------+-------------------+--------------------+--------------+------------+------------+
|stop_id|        stop_name|    arrival_time|  arrival_time_prog|arr_time_prog_status|transport_type|arrival_hour|minute_delay|
+-------+-----------------+----------------+-------------------+--------------------+--------------+------------+------------+
|8502274|          Zufikon|03.09.2018 07:06|03.09.2018 07:06:58|                REAL|           Zug|           7|         0.0|
|8502275|         Heinrüti|03.09.2018 07:10|03.09.2018 07:10:22|                REAL|           Zug|           7|         0.0|
|8502268|Zufikon Belvédère|03.09.2018 07:11|03.09.2018 07:12:00|                REAL|           Zug|           7|         1.0|
+-------+-----------------+----------------+-------------------+--------------------+--------------+------------+------------+
only showing top 3 rows

In [13]:
# Set negative delay to zero
correctNegativeDiff = f.udf(lambda diff: 0 if diff < 0 else diff, FloatType())
delay_df = delay_df.withColumn('minute_delay', correctNegativeDiff(delay_df.minute_delay))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Group and calculate mean and standard deviation

We group according to four columns: stop_id, arrival_hour, transport_type, arr_time_prog_status. Grouping by stop_id gives us all of the unique stop ids, grouping by arrival_hour gives us the desired time intervals, grouping by transport_type is done to see if there is a difference between diferent transport types, and gruping by arr_time_prog_status is done to distinguish the different prognose types.

In [ ]:
# Group the dataframe, calculate mean and std and sort
distribution_df = delay_df.groupBy(['stop_id', 'arrival_hour', 'transport_type', 'arr_time_prog_status'])\
                          .agg(f.mean('minute_delay').alias('mean_minute_delay'), 
                               f.stddev('minute_delay').alias('std_minute_delay'), 
                               f.count('minute_delay').alias('values_count'))\
                          .sort(f.asc('stop_id'), f.asc('arrival_hour'))

distribution_df.cache()
distribution_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Note:** Some groupings don't have many samples, which could potentially lead to statistical insignificance. 

## Calculate probability to be on time

- Delay obliges a gaussian probability distribution.
- being on time means seaching for a probability of having a delay inferior than 0 (or max delay on stop if we have time)
- this means we search for : P(X <= 0) : this is a gaussian integral
- our given parameters : std / mean / number of values

To find the probability that the variable has a value LESS than or equal
let's say 113, you'd use CDF cumulative Density Function

##### scipy.stats.norm.cdf(113,100,12)
Output: 0.86066975255037792
 or 86.07% probability

- Source :https://stackoverflow.com/questions/12412895/calculate-probability-in-normal-distribution-given-mean-std-in-python

loc=mean | scale=std | number1=z where  P( X ≤ z )

In [ ]:
import scipy.stats

# Function to calculate probability
def calc_probability(mean, std): 
    p = scipy.stats.norm.cdf(0, loc=mean, scale=std)
    return float(p)

calc_prob = f.udf(calc_probability, FloatType())

# Add probability for each stop to be on time
distribution_df = distribution_df.withColumn('probability', calc_prob(distribution_df.mean_minute_delay,
                                                                      distribution_df.std_minute_delay))

distribution_df.show(3)

In [ ]:
distribution_df.write.format("parquet").mode('overwrite').save("/user/fristedt/distribution_df.parquet")

==================================================================================================================

## Another idea that we could look at

## Calculate delay interval with confidence percentage as input
- Determine the confidence interval by first finding the standard error:
##### SE = s / √n
- Where s is your sample standard deviation and n is your sample size. For example, if you took a sample of 1,000 men to figure the average weight of a man, and got a sample standard deviation of 30, this would give:
SE = 30 / √1000 = 30 / 31.62 = 0.95
- To find the confidence interval from this, look up the confidence level you want to calculate the interval for in a Z-score table and multiply this value by the Z score. For a 95 percent confidence level, the Z-score is 1.96. Using the example, this means:

#### Mean ± Z × SE= 180 pounds ± 1.96 × 0.95 = 180 ± 1.86 pounds

Here, ± 1.86 pounds is the 95 percent confidence interval.

- If you have this bit of information instead, along with the sample size and the standard deviation, you can calculate the confidence level by using the following formula:

Z = 0.5 × size of confidence interval × √n / s

- The size of the confidence interval is just twice the ± value, so in the example above, we know 0.5 times this is 1.86. This gives:

Z = 1.86 × √1000 / 30 = 1.96

- This gives us a value for Z, which you can look up in a Z-score table to find the corresponding confidence level.

In [88]:
#Confidence level definition

u=0.95
import numpy as np
#Get interval out of confidence level / 95% corresponds to 1,96 z-score

def calc_interval(number, std):
    p=(1.96*std)/math.sqrt(number)
    return float(p)

# User defined function
calc_i = f.udf(calc_interval, FloatType())


diff_m=diff_m.withColumn('time_confidence', calc_i(diff_m.number_of_ech,diff_m.std_d) )

#z-score - confidence
#1,65 - 90%
#2,58 - 99%
diff_m.show(3)

#todo : correcting the code to get float and compute z-score out of confidence percentage

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+--------------+------------------+------------------+-------------+-----------+---------------+
|stop_id|arrival_time_round|transport_type|            mean_d|             std_d|number_of_ech|probability|time_confidence|
+-------+------------------+--------------+------------------+------------------+-------------+-----------+---------------+
|8500926|                 7|           Bus|0.3058918482647296|1.7318332358023905|         1239| 0.42989993|     0.09643318|
|8500926|                 8|           Bus|0.1532258064516129|1.7408557581260544|         1240| 0.46493137|    0.096896484|
|8500926|                 9|           Bus|0.6592292089249493|1.4382381403735474|          986| 0.32334733|     0.08977355|
+-------+------------------+--------------+------------------+------------------+-------------+-----------+---------------+
only showing top 3 rows